<h1 style="font-family:Impact,Arial;font-size:30px;">37004 Interest Rates and Credit Risk Models - Spring 2024</h1>
<h1 style="font-family:Impact,Arial;font-size:45px;">Assignment Part 1</h1>
<h2 style="font-family:Arial;">Erik Schl&ouml;gl</h2>
<p><small> School of Mathematical &amp; Physical Sciences<br>
University of Technology Sydney
</small></p>
<p>
<a href="mailto:Erik.Schlogl@uts.edu.au?Subject=37000 JIT" target="_blank">
<small><font color=MediumVioletRed>Erik.Schlogl@uts.edu.au</font></small></a>
</p>
<hr style="height:5px;border:none;color:#333;background-color:#333;" />


Execute the following cell for a `pandas` DataFrame of Australian interest rate data on 11 August 2023:


In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame([{"Type":"BBSW","Term":"1 month","Rate":4.06},{"Type":"BBSW","Term":"3 months","Rate":4.17},{"Type":"BBSW","Term":"6 months","Rate":4.50},{"Type":"Swap","Term":"1 year","Rate":4.29},{"Type":"Swap","Term":"3 years","Rate":4.06},{"Type":"Swap","Term":"5 years","Rate":4.20},{"Type":"Swap","Term":"10 years","Rate":4.45},{"Type":"Swap","Term":"15 years","Rate":4.58}])
df

,Type,Term,Rate
0,BBSW,1 month,4.06
1,BBSW,3 months,4.17
2,BBSW,6 months,4.50
3,Swap,1 year,4.29
4,Swap,3 years,4.06
5,Swap,5 years,4.20
6,Swap,10 years,4.45
7,Swap,15 years,4.58


Note that the bank bill swap rate (BBSW) is quoted per annum with simple compounding, and "Swaps are quoted on a quarterly basis for maturities out to 3 years and on a semi-annual basis for maturities 4 years and greater." AFMA (2023), Interest Rate Derivative Conventions (see https://afma.com.au/getattachment/Standards/Market-Conventions/Sections/Content/Interest-Rate-Derivative-Conventions-2023-07.pdf?lang=en-AU )


<H2>Task 1:</H2>
Determine the term structure of zero coupon bond prices that these rates imply, using loglinear interpolation where necessary. <I>(4 marks)</I>


In [1]:
# your code here

<H2>Task 2:</H2>
Find the coefficients of a "maximum smoothness" term structure (as defined on Slides 34 to 36 of Lecture 1b) fitting the BBSW and swap rates given above. <I>(10 marks)</I>


In [3]:
# your code here

<H2>Task 3:</H2>
Consider a coupon bond with a principal of 100 and a coupon rate of 8% per annum, where the bond matures in exactly five years and the coupons are paid quarterly. Using the term structure that you derived in Task 1, what is the value of this bond? Similarly, using the term structure that you derived in Task 2, what is the value of this bond? <I>(1 mark)</I>


In [ ]:
# your code here